# Get results based on my snp values

## read in vcf files

In [ ]:
import pickle
from pathlib import Path
from typing import List

import vcf

In [ ]:
cache_file_name = "/home/s/src/search_your_dna/data/vcf_records.pkl"
vcf_file_paths = [Path(f) for f in [
    "/home/s/Dropbox/Siim/health/genetest_2020/GFX0237425.cnv.vcf.gz",
    "/home/s/Dropbox/Siim/health/genetest_2020/GFX0237425.filtered.indel.vcf.gz",
    # "/home/s/Dropbox/Siim/health/genetest_2020/GFX0237425.filtered.snp.vcf.gz",
    "/home/s/Dropbox/Siim/health/genetest_2020/GFX0237425.sv.vcf.gz"
]]

def cache_vcf_records(records: List, overwrite: bool = False) -> None:
    if overwrite and Path(cache_file_name).exists():
        raise Exception("Won't overwrite existing cache file. Please set flag specifically to do that")
    with open(cache_file_name, "wb") as f:
        print(f"Caching records to {cache_file_name}")

        pickle.dump(records, f)

def read_vcf_files(vcf_file_paths):
    if Path(cache_file_name).exists():
        print(f"Reading cached files from {cache_file_name}")
        with open(cache_file_name, 'rb') as f:
            records = pickle.load(f)
    else:
        records = []
        for vcf_file_path in vcf_file_paths:
            print(f"Reading in source vcf file {vcf_file_path}")
            vcf_reader = vcf.Reader(filename=str(vcf_file_path))
            for record in vcf_reader:
                records.append(record)
        cache_vcf_records(records=records)
    return records

records = read_vcf_files(vcf_file_paths=vcf_file_paths)

In [ ]:
len(records)

In [ ]:
print(records[0])

In [ ]:
import mwclient

agent = 'MySNPBot. Run by User:siim. Using mwclient/' + mwclient.__version__
# tokens and secrets are only necessary if your bot will write into SNPedia.
# get your own tokens at http://bots.snpedia.com/index.php/Special:OAuthConsumerRegistration
site = mwclient.Site(('https', 'bots.snpedia.com'), path='/',
                    clients_useragent=agent,
                    consumer_token='???',
                    consumer_secret='???',
                    access_token='???',
                    access_secret='???')


res = site.search("Alzheimer")

In [ ]:
a = site.pages["Alzheimer"]
b = a.resolve_redirect()

In [ ]:
c = b.text()

In [ ]:
print(c)

## create mapping from rsID to alleles

## ask for a search word

## request data from snpedia

## compile a view